Crawling Web Berita Tirto
Crawling adalah proses otomatis untuk mengumpulkan data dari website menggunakan program bernama web crawler.

Import Library
Impor pustaka yang diperlukan seperti :
- requests digunakan untuk mengirim HTTP request ke website
- BeautifulSoup untuk mem-parsing HTML agar bisa mengekstrak data dari halaman web
- pandas untuk mengolah dan menyimpan data dalam bentuk DataFrame.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

Mengambil halaman web
Mengambil halaman web dari URL yang ditentukan dan mempersiapkannya untuk diolah lebih lanjut dengan menggunakan pustaka BeautifulSoup

In [ ]:
#mendefinisikan URL yang menjadi target scraping
main_link = "https://tirto.id/q/latest-news-D5ie"

#mengirimkan request ke URL dan mendapatkan respons
response = requests.get(main_link)

#mem-parsing HTML untuk memudahkan proses ekstraksi data
soup = BeautifulSoup(response.content, 'html.parser')

Inisialisasi List untuk Menyimpan Data
List ini digunakan untuk menyimpan data yang diambil selama proses scraping

In [ ]:
data = []

Mencari bagian yang berisi berita


In [ ]:
articles = soup.find_all('div', class_='list-card list-card-hero')

Melakukan Loop untuk Berita yang Akan Diambil
Melakukan loop atau pengulangan yang bertujuan mengambil informasi dari sejumlah berita yang telah ditemukan di halaman web.

In [ ]:
#jumlah berita yang diambil
num_articles = 10
item = 1

#looping berjalan sesuai dengan banyaknya nilai num_articles, di mana setiap iterasi mengambil satu artikel.
for i in range(num_articles):
      #mengambil berita
      article = articles[i]
      #mengambil judul berita
      title = article.find('h3').text
      #mengambil URL berita
      link = article.find('a')['href']

      #memparsing html
      article_response = requests.get(link)
      article_soup = BeautifulSoup(article_response.content, 'html.parser')

      #mencari elemen yang berisi informasi tentang penulis dan tanggal artikel
      byline_div = article_soup.find('div', class_='byline')
      if byline_div:
          #mencari elemen yang mengandung kata "Terbit", karena merupakan tanggal terbit
          date_div = byline_div.find('div', string=lambda x: x and 'Terbit' in x)
          #membersihkan tanggal dari spasi berlebih
          date_text = date_div.text.strip() if date_div else 'Tanggal tidak ditemukan'
      else:
          #jika tanggal tidak ditemukan, maka akan muncul output 'Tanggal tidak ditemukan'
          date_text = 'Tanggal tidak ditemukan'

      #mencari dan mengekstrak konten spesifik
      content_div = article_soup.find('div', style=lambda value: value and 'font-size:.89em;' in value)
      #memeriksa elemen
      if content_div:
          title_div = content_div.find('div')
          title_text = title_div.text.strip() if title_div else 'Teks tidak ditemukan'

      #mengekstrak teks lengkap dalam bentuk paragraf dan digabungkan menjadi satu untuk pemrosesan lebih lanjut
      content_section = article_soup.find('div', class_='content-text-editor')
      paragraphs = content_section.find_all('p')
      content = "\n".join([p.text for p in paragraphs])

      #menyimpan data yang diambil
      article_data = {
          "Judul": title,
          "Isi": content,
          "Tanggal": date_div,
          "Kategori": title_div
      }
      data.append(article_data)

Menampilkan Data Frame

In [ ]:
df = pd.DataFrame(data)

df

,Judul,Isi,Tanggal,Kategori
0,Ridwan Kamil Bantah Ditolak Warga saat Hadiri ...,"tirto.id - Bakal Calon Gubernur DKI Jakarta, R...","[Terbit 5 Sep 2024 14:52 WIB, ]","[[tirto.id], - Politik]"
1,Antusiasme Warga Mengikuti Misa Agung Paus Fra...,tirto.id - Kunjungan apostolik atau kerasulan ...,"[Terbit 5 Sep 2024 14:48 WIB, ]","[[tirto.id], - Sosial budaya]"
2,"Kejar Target Nol Emisi 2060, Indonesia Garap 4...",tirto.id - Menteri Koordinator Bidang Kemariti...,"[Terbit 5 Sep 2024 14:41 WIB, ]","[Reporter: Qonita Azzahra, [], Penulis: Qonita..."
3,KPU DKI: Visi Misi Ada yang Cuma Selembar dan ...,tirto.id - Komisi Pemilihan Umum (KPU) Jakarta...,"[Terbit 5 Sep 2024 14:40 WIB, ]","[[tirto.id], - Politik]"
4,BKN Perpanjang Pendaftaran CPNS hingga 10 Sept...,tirto.id - Proses pendaftaran seleksi Calon Pe...,"[Terbit 5 Sep 2024 14:39 WIB, ]","[[tirto.id], - Sosial budaya]"
5,Petinggi Vale Temui Presiden Jokowi Bahas Prog...,tirto.id - Presiden Joko Widodo (Jokowi) mener...,"[Terbit 5 Sep 2024 14:34 WIB, ]","[[tirto.id], - Ekonomi]"
6,Transjakarta Buka 2 Depo untuk Parkir Misa Agu...,tirto.id - PT Transportasi Jakarta (Transjakar...,"[Terbit 5 Sep 2024 14:32 WIB, ]","[[tirto.id], - Sosial budaya]"
7,Indonesia Bakal Punya PLTS Terbesar di Dunia S...,tirto.id - Indonesia akan mempunyai Pembangkit...,"[Terbit 5 Sep 2024 14:07 WIB, ]","[[tirto.id], - Ekonomi]"
8,"Faisal Basri Wafat, Luhut: Indonesia Kehilanga...",tirto.id - Menteri Koordinator Bidang Kemariti...,"[Terbit 5 Sep 2024 13:55 WIB, ]","[[tirto.id], - Sosial budaya]"
9,"Paus Fransiskus: Harta Berharga RI Bukan Emas,...",tirto.id - Pemimpin Gereja Katolik sekaligus K...,"[Terbit 5 Sep 2024 13:52 WIB, ]","[[tirto.id], - Sosial budaya]"
